# Latihan-2 Elastic Logs Setup

Latihan ini fokus terhadap data log elastik, pembuatan litofasies sederhana melalui cutoff, pembuatan atribut log elastik seperti impedansi akustik dan rasio vp/vs yang berguna untuk analisis selanjutnya. Fokus berikutnya adalah melakukan blocking, averaging dan crossplot berdasarkan zonasi litofasies
<br><br> _citation for this page: "Adi Widyantoro, 2021, Pertamina Seismic QI Course, Latihan-2 Elastic Logs Setup.ipynb, accessed MM DD, YYYY."_  
<br>
>(update terakhir tanggal 1 Juli 2021 oleh Adi Widyantoro)
<hr>

__Tahap-1__ 
<br> Tetap menggunakan pre-built libraries dasar seperti numpy, matplotlib dan pandas. Sehingga kita tidak perlu membuat semua fungsi yang kita perlukan dari awal/scratch
<br> 
        
        special notes_: brugeslibrary adalah kumpulan pre-defined functions oleh Agile Geoscience bruges:
        https://github.com/agile-geoscience/bruges
        
        the library's credits go to bruges contributors: Evan Bianco, Ben Bougher, Matt Hall, Alessandro Amato del Monte, 
        Wes Hamlyn, Sean Ross-Ross

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from f02qctop import well_sect
from brugeslibrary import backus, moving_average, moving_avg_fft

__Tahap-2__
<br>Menggunakan file lokal atau yang tersimpan online untuk analisis. File path untuk lokal perlu diperhatikan dan diperbaiki jika perlu, menggunakan full path e.g. "C:/Users/Guest/Documents/python/well02logs.csv". <br>Kemudian fungsi selanjutnya melakukan modifikasi header, pengaturan kolom, dan operasi lain, jika memang diperlukan agar mudah dibaca sebagai input analisis berikutnya
<br><br>_catatan_: library pandas (pd) dapat menggunakan tipe file lain seperti excel dengan command pd.read_excel('well02logs.xlsx'). Kemudian atribut fungsi reading file tersebut dapat dipelajari pada website pandas

In [ ]:
# adi2=pd.read_csv('/content/adi02logs.csv') #jika menggunakan google colab' local
adi02=pd.read_csv('adi02logs.csv', warn_bad_lines=True, index_col=0)
adi02.rename(columns={'Depth_m':'Depth', 'Vp_km/s':'Vp', 'Vs_km/s':'Vs','GammaRay_api':'GR',
                      'Density_g/cc':'Rho','Caliper_m':'Cal','Swt_v/v':'Swt','Phit_v/v':'Phit', 
                      'Vsh_v/v':'Vsh'}, inplace=True)

__Tahap-3__
<br>
Menambahkan marker dari tabel untuk pembuatan zonasi. Karena jumlahnya yang sedikit, maka penambhaan marker dpat juga menggunakan array secara manual

In [ ]:
# adi02tops=pd.read_csv('/content/adi02markers.csv') #jika menggunakan google colab' local
adi02tops=pd.read_csv('adi02markers.csv')
adi02tops.rename(columns={'Tops':'marker','Depth_m':'markerdep'}, inplace=True)
marker=np.array(adi02tops['marker'])
markerdep=np.array(adi02tops['markerdep'])
print(adi02tops, " ", sep='\n')
printmarker=marker.transpose()
printmarkerdep=markerdep.transpose()
print ('markers:', printmarker)

__Tahap-4__
<br>
Menampilkan tops dengan membuat fungsi plotting well section yang dapat digunakan agar fokus terhadap interval of interest saja.
<br><br>
Ubah ekstensi interval of interest sesuai instruksi pada saat kursus, dengan mengubah nilai _top_plot_, _base_plot_
<br><br> _catatan_: jika mengalami masalah dalam menjalankan command line di bawah dapat dilakukan dengan menggunakan cmd line sbb: exec(open('02_qctop.py').read())

In [ ]:
datawell=adi02
top_plot=1650
base_plot=1900
marker=marker
markerdepth=markerdep
well_sect(datawell,top_plot,base_plot,marker,markerdepth)

__Tahap-5__
<br>
Membuat zonasi litofasies, berdasarkan marker, cutoff vsh, porositas, saturasi. <br>
Misalnya:

|Code |Description| Flag         
|:-   |---------: | :-:
|oil  |oil sand   | 1
|ssd  |shaley sand| 2
|sh   |shales     | 3
|bri  |brine sand | 4
|coal |coal       | 5

In [ ]:
pd.options.mode.chained_assignment = None
top_plot=1650
base_plot=1800
logs=adi02[(adi02.index >= top_plot) & (adi02.index <= base_plot)]
sand=0.3
shale=0.7
pay=0.85
coal =1.75
oil=((logs.Vsh <= sand) & (logs.Swt <= pay))
ssd=((logs.Vsh < shale) & (logs.Vsh > sand))
sh=((logs.Vsh >= shale))
brine=((logs.Vsh <= sand) & (logs.Swt > pay))
coal=((logs.Vsh <= sand) & (logs.Rho < coal))
flag=np.zeros(np.shape(logs.Vsh))
flag[oil.values]=1
flag[ssd.values]=2
flag[sh.values]=3
flag[brine.values]=4
flag[coal.values]=5
logs['fac']=flag

In [ ]:
print(logs.fac.head(20))

__Tahap-6__
<br>
Menampilkan histogram log elastik untuk masing-masing fasies. Statistik yang dihasilkan dapat digunakan sebagai input half space model, e.g. layered atau blocky AVO

In [ ]:
exec(open('f02showhistogram.py').read())
# plt.savefig('02_elasticlogs_013.png', dpi=200)

In [ ]:
import matplotlib.colors as colors
warna = ['gray','green','orange','olive','blue', 'black']
skala_fasies = colors.ListedColormap(warna[0:len(warna)], 'indexed')
plt.figure(figsize=(6,4))
plt.scatter(logs.Vp,logs.Rho,80,logs.fac,marker='o',
            edgecolors='none',alpha=0.5,cmap=skala_fasies,vmin=0,vmax=5)
plt.xlim(2,4.6); plt.ylim(1,3); plt.grid(color='grey', linestyle=':'); 
plt.xlabel('Vp', size=20); plt.ylabel('Rho', size=20)
cbar=plt.colorbar()
cbar.set_label((6*' ').join(['n/a','oil', 'ssd', 'sh', 'brine','coal']))
cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')

__Tahap-7__
<br>
Menghitung log moduli atau log elastik lain yang diperlukan seperti impedansi akustik, impedansi elastik, vp/vs rasio dsb

In [ ]:
logs['Vp']=logs.Vp
logs['Vs']=logs.Vs
logs['Rho']=logs.Rho
logs['AI']=round((logs.Vp*logs.Rho),2)
logs['SI']=round((logs.Vs*logs.Rho),2)
logs['VpVs']=round((logs.Vp/logs.Vs),2)
print(logs.head(10))

In [ ]:
print('AI min: {}, AI max: {}'.format(logs.AI.min(), logs.AI.max()))
print('SI min: {}, SI max: {}'.format(logs.SI.min(), logs.SI.max()))
print('VpVs min: {}, VpVs max: {}'.format(logs.VpVs.min(), logs.VpVs.max()))

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=3, figsize=(14, 4))
ax[0].scatter(logs.Phit,logs.Vp,80,logs.fac,marker='o',edgecolors='none',alpha=0.5,cmap=skala_fasies,vmin=0,vmax=5)
ax[1].scatter(logs.AI,logs.SI,80,logs.fac,marker='o',edgecolors='none',alpha=0.5,cmap=skala_fasies,vmin=0,vmax=5)
ax[2].scatter(logs.AI,logs.VpVs,80,logs.fac,marker='o',edgecolors='none',alpha=0.5,cmap=skala_fasies,vmin=0,vmax=5)
ax[0].set_xlim(0,0.3); ax[0].set_ylim(2,4.6);
ax[1].set_xlim(2,12); ax[1].set_ylim(0.5,8);
ax[2].set_xlim(2,12); ax[2].set_ylim(0.5,3);
ax[0].set_title('Vp-Phit', size=20), ax[0].set_xlabel('Phit', size=14), ax[0].set_ylabel('Vp', size=14);
ax[1].set_title('AI-SI', size=20), ax[1].set_xlabel('AI', size=14), ax[1].set_ylabel('SI', size=14);
ax[2].set_title('AI-Vp/Vs', size=20), ax[2].set_xlabel('AI', size=14), ax[2].set_ylabel('VpVs', size=14);
for i in ax: i.grid(color='grey', linestyle=':')

<h1><center>-Akhir Program Latihan-2-<center></h1>